<a href="https://colab.research.google.com/github/bhumong/ai-bootcamp/blob/main/practice/nlp1_rg_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dummy Intuition

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Contoh data sederhana (analog dengan input dan target pada encoder-decoder)
input_teks = [
    "Saya suka film ini",  # Input: kalimat yang akan di-"encode"
    "Cuaca hari ini cerah",
    "Saya merasa sedih"
]
target_label = ['positif', 'positif', 'negatif']  # Target: analog dengan "output" yang diharapkan dari decoder

In [ ]:
# 1. Encoder (diwakili oleh TF-IDF Vectorizer)
#    - Mengubah teks input menjadi representasi numerik (vektor)
encoder = TfidfVectorizer()
encoder.fit(input_teks)  # "Melatih" encoder dengan vocabulary dari input
encoded_input = encoder.transform(input_teks)  # Meng-"encode" input menjadi matriks TF-IDF

print("Encoded Input (TF-IDF Matrix):")
print(encoded_input.toarray())  # Menampilkan representasi numerik dari input

Encoded Input (TF-IDF Matrix):
[[0.         0.         0.5628291  0.         0.42804604 0.
  0.42804604 0.         0.5628291 ]
 [0.52863461 0.52863461 0.         0.52863461 0.40204024 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.62276601
  0.4736296  0.62276601 0.        ]]


In [ ]:
# 2. Decoder (diwakili oleh Naive Bayes Classifier)
#    - Menerima representasi numerik dari encoder dan memprediksi label
#    - Dalam contoh ini, decoder "dilatih" untuk memetakan vektor TF-IDF ke label sentimen
teks_latih, teks_uji, label_latih, label_uji = train_test_split(encoded_input, target_label, test_size=0.2, random_state=42)

decoder = MultinomialNB()
decoder.fit(teks_latih, label_latih)  # Melatih decoder dengan data yang sudah di-encode

MultinomialNB()

In [ ]:
# Contoh penggunaan decoder untuk memprediksi label dari teks baru
teks_baru = ["Saya sangat senang"]
encoded_teks_baru = encoder.transform(teks_baru)  # Encode teks baru menggunakan encoder yang sudah dilatih
prediksi = decoder.predict(encoded_teks_baru)  # Decoder memprediksi label

print(f"Prediksi sentimen untuk teks baru: {prediksi}")

Prediksi sentimen untuk teks baru: ['negatif']


**Penjelasan:**

*   **Encoder (diwakili oleh `TfidfVectorizer`)**: Dalam kode ini, `TfidfVectorizer` berfungsi sebagai *encoder*. Ia mengambil teks input dan mengubahnya menjadi matriks TF-IDF, yang merupakan representasi numerik dari teks. Proses ini analog dengan *encoder* yang mengubah bahasa manusia menjadi format yang dapat dipahami oleh mesin.  `encoder.fit(input_teks)` melatih *encoder* dengan *vocabulary* dari data input.
*   **Decoder (diwakili oleh `MultinomialNB`)**: `MultinomialNB` berfungsi sebagai *decoder*. Ia menerima output dari *encoder* (matriks TF-IDF) dan memprediksi label yang sesuai (dalam hal ini, sentimen). *Decoder* "dilatih" untuk memetakan representasi numerik dari *encoder* ke label yang diinginkan.
*   **Alur Kerja**:
    1.  Teks input di-"*encode*" menjadi matriks TF-IDF menggunakan `TfidfVectorizer` (***encoder***).
    2.  Matriks TF-IDF ini kemudian digunakan untuk melatih `MultinomialNB` (***decoder***) untuk memprediksi sentimen.
    3.  Untuk teks baru, teks tersebut pertama-tama di-*encode* menggunakan `TfidfVectorizer` yang sudah dilatih, dan kemudian *decoder* memprediksi sentimennya.

**Analogi dengan Konsep NLP:**

*   Encoder mengubah kalimat menjadi vektor fitur.
*   Decoder menggunakan vektor fitur ini untuk membuat prediksi.
*   Proses ini mencerminkan intuisi dasar dari *encoder-decoder* model, di mana *encoder* memahami input dan *decoder* menghasilkan output berdasarkan pemahaman tersebut.

**Penting untuk diingat:**

*   Ini adalah penyederhanaan yang signifikan. Model *encoder-decoder* modern (seperti yang digunakan dalam *transformers*) jauh lebih kompleks dan menggunakan *neural networks* untuk *encoding* dan *decoding*. Contoh ini hanya untuk memberikan intuisi dasar menggunakan alat yang lebih sederhana.
*   Kode ini tidak menggunakan *word embedding* secara eksplisit, tetapi TF-IDF dapat dianggap sebagai bentuk representasi kata numerik.
*   *Preprocessing* (seperti yang dijelaskan dalam sumber) akan meningkatkan kinerja model ini secara signifikan.

Kode di atas memberikan ilustrasi sederhana tentang bagaimana prinsip *encoder* dan *decoder* dapat diimplementasikan menggunakan `sklearn`, meskipun dengan keterbatasan yang signifikan dibandingkan dengan model NLP modern.

### Imports

In [ ]:
import numpy as np
import pandas as pd
import torch
import random
import re
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Read Data

## Clean

In [ ]:
stopword = []
with open('/content/drive/MyDrive/project-absa-new/clean/stopwords.txt', 'r') as file:
    for word in file:
        stopword.append(word.strip())

# import requests
# url = "https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/refs/heads/master/stopwords_id_satya.txt"

# response = requests.get(url)
# stopword = response.text.splitlines()  # Memisahkan setiap baris menjadi elemen list

slangwords = {}
with open('/content/drive/MyDrive/project-absa-new/clean/slangwords.txt', 'r') as file:
    for line in file:
        words = line.split(",")
        old = words[0].strip()
        new = words[1].strip()
        slangwords[old] = new

# import requests
# url = "https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/refs/heads/master/combined_slang_words.txt"
# response = requests.get(url)
# lines = response.text.splitlines()

# slangwords = {}
# for line in lines:
#     words = line.split(",")
#     if len(words) == 2:  # Pastikan ada pasangan old → new
#         old, new = words[0].strip(), words[1].strip()
#         slangwords[old] = new

def convertToSlangword(review):
    review = review.split()
    content = []
    for kata in review:
        if kata in slangwords:
            new_kata = slangwords[kata]
        else:
            new_kata = kata
        content.append(new_kata.lower())
    return ' '.join(content)

def filtering(review, remove_numbers=True):
    # Menghapus URL
    review = re.sub(r'https?://\S+', ' ', review)

    review = re.sub(r'@[\w\.]+\b', ' ', review)
    review = re.sub(r'@\w+', ' ', review)

    review = re.sub(r"[.,:;]", " ", review)

    # Menghapus kata setelah tanda pagar (#) hanya jika jumlah hashtag tepat 3
    hashtags = re.findall(r'#([^\s]+)', review)
    for hashtag in hashtags:
        review = re.sub(r'#' + re.escape(hashtag) + r'\b', ' ', review)

    review = re.sub(r"\d", " ", review) if remove_numbers else review
    review = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\*]", " ", review)
    review = re.sub(r'[^\x00-\x7f]', r' ', review)
    review = re.sub(r'(\\u[0-9A-Fa-f]+)', r' ', review)
    review = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", review)
    review = re.sub(r'\\u\w\w\w\w', ' ', review)
    review = re.sub(r'@\w+\b', ' ', review)
    review = re.sub(r'\s+', ' ', review)

    # remove some words
    rmv = ['a', 'href', 'quot']
    review = word_tokenize(review)
    review = [str(token).strip() for token in review if token not in rmv]
    review = ' '.join(review)

    return review.strip()

def replaceThreeOrMore(review):
        pattern = re.compile(r"(\w)\1{2,}")
        return pattern.sub(r"\1", review)

def remove_stopwords(text):
    text = text.split()
    text = [token for token in text if token not in stopword]
    return ' '.join(text)

In [ ]:
def clean(review, slang=True, sw=True, num=True):
    review = review.lower() #casefolding
    review = filtering(review, remove_numbers=num)
    review = replaceThreeOrMore(review)
    review = convertToSlangword(review) if slang else review
    review = remove_stopwords(review) if sw else review
    return review

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df.duplicated().sum()

13407

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.content.isna().sum()

0

In [ ]:
df = df.dropna(subset=['content']).reset_index(drop=True)

In [ ]:
df['clean'] = df['content'].apply(lambda x: clean(x))

In [ ]:
df

In [ ]:
df.duplicated().sum()

0

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.isna().sum()

,0
aspect,0
content,0
final_sentiment,0
clean,0


In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
len(df)

97782